In [ ]:
# Marvel Model

In [ ]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import pickle
import re
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
import random

In [ ]:
# read pickle file
pickle_in = open("marvel_list.pkl","rb")
movie_plots = pickle.load(pickle_in)

# count of movie plot summaries
len(movie_plots)

1

In [ ]:
movie_plots[0]

'Investigating the murder of a teenage girl Cage suddenly learns that a threeway gang war is under way for control of the turf 3D Man Acts of Vengeance Loki sets about convincing the supervillains of Earth to attack heroes other than those they normally fight in an attempt to destroy the Avengers to absolve his guilt over inadvertently creating the team in the first place In the wake of September 11th the world watched as firefighters police officers and EMT workers selflessly risked their lives ABomb HAS Rick Jones has been Hulks best bud since day one but now hes more than a friendhes a teammate Transformed by a Gamma energy explosion ABombs thick armored skin is just as strong and powerful as it is blue And when he curls into action he uses it like a giant bowling ball of destruction Age of Apocalypse In a twisted version of the world they knew the XMen battle against the eternal mutant Apocalypse as Bishop seeks to repair the timeline Legion Xaviers own son attempts to kill off all

In [ ]:
def get_fixed_sequence(text, seq_len = 5):
  sequences = []
  words = text.split()
  if len(words) > seq_len:
    for i in range(seq_len, len(words)):
      seq_list = words[i-seq_len: i]
      sequences.append(" ".join(seq_list))
  else:
    sequences = words
  return sequences

In [ ]:
seqs = [get_fixed_sequence(plot) for plot in movie_plots]

In [ ]:
len(seqs)

1

In [ ]:
seqs = sum(seqs, [])

In [ ]:
seqs[1]

'the murder of a teenage'

In [ ]:
x = []
y = []
for seq in seqs:
  words = seq.split()
  x.append(" ".join(words[:-1]))
  y.append(" ".join(words[1:]))

In [ ]:
x[0], y[0]

('Investigating the murder of', 'the murder of a')

In [ ]:
# create integer-to-token mapping
int2token = {}
cnt = 0

for w in set(" ".join(movie_plots).split()):
  int2token[cnt] = w
  cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

token2int["the"], int2token[71]

(523, 'us')

In [ ]:
token2int

{'1': 715,
 '10issue': 637,
 '11': 61,
 '11th': 154,
 '12issue': 524,
 '2017': 298,
 '3D': 579,
 'A': 586,
 'ABomb': 249,
 'ABombs': 236,
 'AIM': 95,
 'Aaron': 225,
 'Abomination': 778,
 'Absorbing': 250,
 'Abyss': 107,
 'Accuser': 762,
 'Acts': 581,
 'Adam': 403,
 'Aegis': 480,
 'After': 515,
 'Age': 740,
 'Agent': 279,
 'Agents': 47,
 'Aginar': 600,
 'Ahab': 664,
 'AirWalker': 201,
 'Ajak': 48,
 'Ajaxis': 455,
 'AllDifferent': 336,
 'AllNew': 472,
 'Amadeus': 219,
 'America': 404,
 'An': 200,
 'And': 660,
 'Andrea': 462,
 'Annihilation': 738,
 'Annihilus': 812,
 'Anything': 157,
 'Apocalypse': 804,
 'Armor': 314,
 'As': 734,
 'Assassin': 665,
 'Atlantis': 108,
 'Atlas': 530,
 'Attacks': 59,
 'Avengers': 374,
 'Axis': 265,
 'Banner': 448,
 'Beehive': 231,
 'Bendis': 187,
 'Bishop': 126,
 'Black': 301,
 'Blockbuster': 699,
 'Blonsky': 824,
 'Blood': 99,
 'Brand': 556,
 'Brandon': 732,
 'Brian': 339,
 'Brubaker': 780,
 'Bruce': 532,
 'Bryan': 125,
 'But': 502,
 'Cabal': 317,
 'Cage': 60

In [ ]:
# set vocabulary size
vocab_size = len(int2token)
vocab_size

826

In [ ]:
def get_integer_seq(seq):
  return [token2int[w] for w in seq.split()]

# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x]
y_int = [get_integer_seq(i) for i in y]

# convert lists to numpy arrays
x_int = np.array(x_int)
y_int = np.array(y_int)

In [ ]:
x_int[0]

array([328, 523,  49,  37])

In [ ]:
def get_batches(arr_x, arr_y, batch_size):
  prev = 0
  for n in range(batch_size, arr_x.shape[0], batch_size):
    x = arr_x[prev:n]
    y = arr_y[prev:n]
    prev = n
    yield x,y

In [ ]:
# Get random words for word bank
random_tokens = random.sample(list(token2int), 30)
random_tokens

['be',
 'This',
 'a',
 'Investigating',
 'Kubert',
 'working',
 'were',
 'death',
 'was',
 'Black',
 'murder',
 'Ronan',
 'Theyve',
 'remaking',
 'Blood',
 'ABombs',
 'series',
 'for',
 'control',
 'bowling',
 'Herc',
 'nightmares',
 'close',
 'Secret',
 'rag',
 'Transformed',
 'can',
 'Hercules',
 'order',
 'argument']

In [ ]:
class WordLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=4, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 200)

        ## define the LSTM
        self.lstm = nn.LSTM(200, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        
        #out = out.contiguous().view(-1, self.n_hidden) 
        out = out.reshape(-1, self.n_hidden) 

        ## put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [ ]:
# instantiate the model
net = WordLSTM()

# push the model to GPU (avoid it if you are not using the GPU)
net.cuda()

print(net)

WordLSTM(
  (emb_layer): Embedding(826, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=826, bias=True)
)


In [ ]:
# Initialize optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
emb_layer.weight 	 torch.Size([826, 200])
lstm.weight_ih_l0 	 torch.Size([1024, 200])
lstm.weight_hh_l0 	 torch.Size([1024, 256])
lstm.bias_ih_l0 	 torch.Size([1024])
lstm.bias_hh_l0 	 torch.Size([1024])
lstm.weight_ih_l1 	 torch.Size([1024, 256])
lstm.weight_hh_l1 	 torch.Size([1024, 256])
lstm.bias_ih_l1 	 torch.Size([1024])
lstm.bias_hh_l1 	 torch.Size([1024])
lstm.weight_ih_l2 	 torch.Size([1024, 256])
lstm.weight_hh_l2 	 torch.Size([1024, 256])
lstm.bias_ih_l2 	 torch.Size([1024])
lstm.bias_hh_l2 	 torch.Size([1024])
lstm.weight_ih_l3 	 torch.Size([1024, 256])
lstm.weight_hh_l3 	 torch.Size([1024, 256])
lstm.bias_ih_l3 	 torch.Size([1024])
lstm.bias_hh_l3 	 torch.Size([1024])
fc.weight 	 torch.Size([826, 256])
fc.bias 	 torch.Size([826])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]}]


In [ ]:
def train(net, epochs=10, batch_size=100, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()
    
    # push model to GPU
    net.cuda()
    
    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(x_int, y_int, batch_size):
            counter+= 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter))

In [ ]:
train(net, batch_size = 100, epochs=20, print_every=256)

Epoch: 18/20... Step: 256...


In [ ]:
def predict(net, tkn, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = net(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h



In [ ]:

# function to generate text
def sample(net, size, prime='The'):
        
    # push to GPU
    net.cuda()
    
    net.eval()

    # batch size is 1
    h = net.init_hidden(1)

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(net, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(net, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)

In [ ]:
sample(net, 5)

'the and the Captain A Stark'

In [ ]:
import pickle
# save the model to disk
filename = 'Marvel_model.pickle'
pickle.dump(net, open(filename, 'wb'))

In [ ]:
# Specify a path
PATH = "Marvel_model.pt"

# Save
torch.save(net, PATH)

In [ ]:
# *** RANDOM WORDS ***

In [ ]:
# **** Option 1: Get random words for word bank without removing common English words ****
random_tokens = random.sample(list(token2int), 50)
random_tokens

In [ ]:
# *** Option2: Get random words from list of words that doesn't include most common English words ***

In [ ]:
# ***** Remove common words from word bank *****

In [ ]:
# read pickle file - most common english words
pickle_in = open("common_words.pkl","rb")
common_words = pickle.load(pickle_in)

# count of movie plot summaries
len(common_words)

In [ ]:
new_movie_list2 = [i for i in token2int if i not in common_words]

In [ ]:
# Get random words for word bank
random_tokens2 = random.sample(list(new_movie_list2), 50)
random_tokens2